In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors
import seaborn as sns
from mne.decoding import Vectorizer, SlidingEstimator, cross_val_multiscore
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [ ]:
# Load data from animal to train classifier
animal = 'Ent_CamK2_15'
idx_animal = 0
list_channels = np.load('Mice_data/'+animal+'_list_channels.npy')
raw_EEG_response = pd.read_pickle('Mice_data/'+animal+'_All_first_pulse_Max Intensity')

# Settings
time_window = 250
Sampling_freq = 2000
timepoints = int(1.5 * time_window * Sampling_freq/1000)
conditions_to_classify = ['NaCl','Dz 5mg/kg','PTZ 20mg/kg']
n_cv = 5 # number of folds for cross validation
n_perm=100 # number of permutations to compute empirical chance level

In [16]:
# Array to score scores
score_tp_per_animal = np.zeros((1,timepoints))
chancelevel_tp_per_animal = np.zeros((1,timepoints))
p_value_tp_per_animal  = np.zeros((1,timepoints))

# Prepare data and labels
data = np.array(raw_EEG_response['LFP'].to_list())  # Dimensions: n_trials x n_channels x n_timepoints
labels = np.array(raw_EEG_response['Condition'].to_list()).flatten()

# Keep only conditions to classify ####
data_cnd = data[np.isin(labels,conditions_to_classify)]
labels_cnd = labels[np.isin(labels,conditions_to_classify)]

# Classifier
clf_tp = make_pipeline(Vectorizer(), StandardScaler(),LogisticRegressionCV(solver='newton-cg', cv=n_cv, random_state=42, penalty='l2'))

# Run classifier at each timepoing using a sliding window
sl = SlidingEstimator(clf_tp, scoring=make_scorer(accuracy_score))
scores_all_folds = cross_val_multiscore(sl, data_cnd, labels_cnd, cv=n_cv, n_jobs=12)

# Mean the score for each fold across the cross validation iterations
score_cv = np.mean(scores_all_folds, axis=0)
score_tp_per_animal[idx_animal] = score_cv

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.










  0%|          | Fitting SlidingEstimator : 0/750 [00:00<?,       ?it/s]

  0%|          | Fitting SlidingEstimator : 1/750 [00:00<01:04,   11.66it/s]
  0%|          | Fitting SlidingEstimator : 1/750 [00:00<01:05,   11.42it/s]


  0%|          | Fitting SlidingEstimator : 1/750 [00:00<01:05,   11.38it/s]



  0%|          | Fitting SlidingEstimator : 1/750 [00:00<01:08,   11.00it/s]

  0%|          | Fitting SlidingEstimator : 2/750 [00:00<00:46,   16.06it/s]






  0%|          | Fitting SlidingEstimator : 2/750 [00:00<00:46,   15.98it/s]
  0%|          | Fitting SlidingEstimator : 2/750 [00:00<00:46,   15.94it/s]
  0%|          | Fitting SlidingEstimator : 3/750 [00:00<00:39,   18.96it/s]





  0%|          | Fitting SlidingEstimator : 3/750 [00:00<00:39,   18.70it/s]


  1%|          | Fitting SlidingEstimator : 4/750 [00:00<00:36,   20.20it/s]
  1%|          | Fitting SlidingEstimator : 4/750 

In [ ]:
# Calculate empricial chance level by permutating the test labels
score_perm = np.zeros((n_perm, data.shape[-1]))
for i in range(n_perm):
    perm_labels_train = np.copy(labels_cnd)
    np.random.shuffle(perm_labels_train)

    sl_perm = SlidingEstimator(clf_tp, scoring=make_scorer(accuracy_score))
    scores_all_data = cross_val_multiscore(sl_perm, data_cnd, perm_labels_train, cv=n_cv, n_jobs=12)
    scores_mean = np.mean(scores_all_data, axis=0)
    score_perm[i] = scores_mean

emprical_chance = np.mean(score_perm, axis=0)
chancelevel_tp_per_animal[idx_animal] = emprical_chance

In [ ]:
#  Calculate p-value as (C + 1) / (n_permutations + 1):
p_value_tp = np.zeros((len(score_cv)))
for tp in range(len(score_cv)):
    C = len(np.where(score_perm[:, tp] >= score_cv[tp])[0])
    p_value_tp[tp] = (C + 1) / (score_perm.shape[0] + 1)

p_value_tp_per_animal[idx_animal] = p_value_tp    df = pd.DataFrame({"Animal": animals, "f1-score": score_tp_per_animal.tolist(), 'chance level': chancelevel_tp_per_animal.tolist(),'p-value': p_value_tp_per_animal.tolist(),'Condition': [gl.list_to_string(hue_order)] * len(animals),'Settings': [[n_cv, n_perm]] * len(animals)})


In [ ]:
result_df = pd.DataFrame({"Animal": animal, "Accuracy": score_tp_per_animal.tolist(), 'chance level': chancelevel_tp_per_animal.tolist(),'p-value': p_value_tp_per_animal.tolist(),'Condition': [gl.list_to_string(hue_order)] * len(animals),'Settings': [[n_cv, n_perm]] * len(animals)})